<img src="https://raw.githubusercontent.com/paular143/images_notebooks/master/logo-sin-nombre.png" alt="Universidad de Los Andes" style="width: 90px;" align="left" frameborder="300"/>

**UNIVERSIDAD DE LOS ANDES** <br>
**FACULTAD DE ECONOMÍA - EDUCACIÓN CONTINUADA** <br>
**Análisis espacial de datos y sus aplicaciones en Python 2020** <br>
**PROFESORES:** Germán González y Mateo Dulce <br><br>


# Clase 5: Golocalización

- Geoencoding



In [1]:
# !pip3 install shapely
# !pip3 install pyproj
# !pip3 install folium
# !pip3 install pandas
# !pip3 install requests
# !pip3 install geopy
# !pip3 install geojson
# !pip3 install xlrd
import pyproj
from functools import partial
import shapely.geometry
from shapely.ops import transform
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
import folium
import geojson
import json
import pandas as pd
import re
import geopy
import requests
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

/Users/germangonzalez/.pyenv/versions/3.7.5/envs/motus/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Cargamos un excel que contiene las direcciones y los nombres de hospedajes en Bogotá

In [2]:
Alojamientos_hospedaje = pd.read_excel('Datos/Direcciones.xlsx')

In [3]:
Alojamientos_hospedaje

,Direccion,Hospedaje
0,Carrera 64 103C 16,103 WONDERFULL HOUSE
1,Calle 51A 74 20,A BOGOTA ON HOLIDAYS
2,Carrera 48 93 64,ALPES BOGOTA HOSTAL
3,Calle 10 2 40,APARTAESTUDIOS LA CANDELARIA
4,Calle 57 18 26,CASA COLONIAL INN
...,...,...
408,Carrera 63 98 66,WONDERFULL HOUSE 100
409,Calle 11 0 38,DESTINO NOMADA HOSTEL
410,Calle 12F 2 38,HOSTAL LA CANDELARIA BOGOTA
411,Calle 12F 2 55,HOSTAL SUE E.U.


## Definimos latitud y longitud

In [4]:
Alojamientos_hospedaje['Latitud'] = None
Alojamientos_hospedaje['Longitud'] = None

In [5]:
Alojamientos_hospedaje

,Direccion,Hospedaje,Latitud,Longitud
0,Carrera 64 103C 16,103 WONDERFULL HOUSE,None,None
1,Calle 51A 74 20,A BOGOTA ON HOLIDAYS,None,None
2,Carrera 48 93 64,ALPES BOGOTA HOSTAL,None,None
3,Calle 10 2 40,APARTAESTUDIOS LA CANDELARIA,None,None
4,Calle 57 18 26,CASA COLONIAL INN,None,None
...,...,...,...,...
408,Carrera 63 98 66,WONDERFULL HOUSE 100,None,None
409,Calle 11 0 38,DESTINO NOMADA HOSTEL,None,None
410,Calle 12F 2 38,HOSTAL LA CANDELARIA BOGOTA,None,None
411,Calle 12F 2 55,HOSTAL SUE E.U.,None,None


# Forma Manual 

## Para una dirección

Para crear las credenciales de HERE se deben registrar en https://developer.here.com/login

Más información de conexiones: https://developer.here.com/blog/understanding-geocoding-with-python 

In [6]:
api_here = '5ELMuTthvjz1Blo6MzU2yhUqD7Jig9tBX4bDrz247-Y'

In [7]:
Direccion = Alojamientos_hospedaje['Direccion'].iloc[0]
busqueda = Direccion + ', Bogotá, Colombia'
PARAMS = {'apikey':api_here,'q':busqueda} 

In [8]:
Direccion = Alojamientos_hospedaje['Direccion'].iloc[0]
busqueda = Direccion + ', Bogotá, Colombia'
PARAMS = {'apikey':api_here,'q':busqueda} 

# Construir solicitud
URL = "https://geocode.search.hereapi.com/v1/geocode"
r = requests.get(url = URL, params = PARAMS) 
data = r.json()

latitude = data['items'][0]['position']['lat']
longitude = data['items'][0]['position']['lng']

In [9]:
for j in range(0,10):    

    try: 
        Direccion = Alojamientos_hospedaje['Direccion'].iloc[j]
        busqueda = Direccion + ', Bogotá, Colombia'
        PARAMS = {'apikey':api_here,'q':busqueda} 
        print('Geocoding de la dirección: '+ busqueda)
        ####### --- Enviar la solicitud -- #### 
        URL = "https://geocode.search.hereapi.com/v1/geocode"
        r = requests.get(url = URL, params = PARAMS) 
        data = r.json()

        latitud = data['items'][0]['position']['lat']
        longitud = data['items'][0]['position']['lng']
    except: 
        latitud = None
        longitud = None
    
    ###### ----- Vamos a sobre escribir ----- #### 
    Alojamientos_hospedaje['Latitud'].iloc[j] = latitud
    Alojamientos_hospedaje['Longitud'].iloc[j] = longitud

Geocoding de la dirección: Carrera 64 103C 16, Bogotá, Colombia
Geocoding de la dirección: Calle 51A 74 20, Bogotá, Colombia
Geocoding de la dirección: Carrera 48 93 64, Bogotá, Colombia
Geocoding de la dirección: Calle 10 2 40, Bogotá, Colombia
Geocoding de la dirección: Calle 57 18 26, Bogotá, Colombia
Geocoding de la dirección: Carrera 18 124 50, Bogotá, Colombia
Geocoding de la dirección: Carrera 12 119 26, Bogotá, Colombia
Geocoding de la dirección: Calle 93 18 11, Bogotá, Colombia
Geocoding de la dirección: Calle 74 10 33, Bogotá, Colombia
Geocoding de la dirección: Carrera 10 27 51, Bogotá, Colombia


In [10]:
Alojamientos_hospedaje

,Direccion,Hospedaje,Latitud,Longitud
0,Carrera 64 103C 16,103 WONDERFULL HOUSE,4.69179,-74.0672
1,Calle 51A 74 20,A BOGOTA ON HOLIDAYS,4.67247,-74.1108
2,Carrera 48 93 64,ALPES BOGOTA HOSTAL,4.6814,-74.0612
3,Calle 10 2 40,APARTAESTUDIOS LA CANDELARIA,4.59482,-74.0718
4,Calle 57 18 26,CASA COLONIAL INN,4.64484,-74.0696
...,...,...,...,...
408,Carrera 63 98 66,WONDERFULL HOUSE 100,None,None
409,Calle 11 0 38,DESTINO NOMADA HOSTEL,None,None
410,Calle 12F 2 38,HOSTAL LA CANDELARIA BOGOTA,None,None
411,Calle 12F 2 55,HOSTAL SUE E.U.,None,None


# Forma automática: Geopy

## Para una dirección

In [11]:
Direccion = 'Carrera 64 103C 16'
busqueda = Direccion + ', Bogotá, Colombia'



In [12]:
### --- Se define el motor de geocoder que se va utilizar: En este caso se utilizara HERE ----- #### 
motor_busqueda = geopy.geocoders.Here(app_id=None, app_code=None, apikey=api_here)
Resultado_busqueda = motor_busqueda.geocode(busqueda,timeout = 10000)

Resultado

In [13]:
Resultado_busqueda

Location(Carrera 64 103C-16, 111121 Bogotá, Colombia, Bogotá 111121, COL, (4.69179, -74.0672, 0.0))

## Busquéda inversa: Dirección a partir de una latitud, longitud

In [14]:
cordenadas_buscar = str(Resultado_busqueda.latitude)+','+str(Resultado_busqueda.longitude)

In [15]:
cordenadas_buscar

'4.69179,-74.0672'

In [16]:
motor_busqueda.reverse(cordenadas_buscar)

Location(Carrera 64 103C-16, 111121 Bogotá, D.C., Colombia, Bogotá, D.C. 111121, COL, (4.69181, -74.06725, 0.0))

# Para las 10 primeras

In [17]:
#### ---- El motor de busqueda solo se tiene que definir una vez --- ### 
motor_busqueda = geopy.geocoders.Here(app_id=None, app_code=None, apikey=api_here)

for j in range(0,10):    
    Direccion = Alojamientos_hospedaje['Direccion'].iloc[j]
    busqueda = Direccion + ', Bogotá, Colombia'
    print(busqueda)
    #### ---- Se realiza la busqueda --- ### 
    Resultado_busqueda = motor_busqueda.geocode(busqueda,timeout = 10000)

    ###### ----- Vamos a sobre escribir ----- #### 
    Alojamientos_hospedaje['Latitud'].iloc[j] = Resultado_busqueda.latitude
    Alojamientos_hospedaje['Longitud'].iloc[j] = Resultado_busqueda.longitude

Carrera 64 103C 16, Bogotá, Colombia
Calle 51A 74 20, Bogotá, Colombia
Carrera 48 93 64, Bogotá, Colombia
Calle 10 2 40, Bogotá, Colombia
Calle 57 18 26, Bogotá, Colombia
Carrera 18 124 50, Bogotá, Colombia
Carrera 12 119 26, Bogotá, Colombia
Calle 93 18 11, Bogotá, Colombia
Calle 74 10 33, Bogotá, Colombia
Carrera 10 27 51, Bogotá, Colombia


In [18]:
Alojamientos_hospedaje

,Direccion,Hospedaje,Latitud,Longitud
0,Carrera 64 103C 16,103 WONDERFULL HOUSE,4.69179,-74.0672
1,Calle 51A 74 20,A BOGOTA ON HOLIDAYS,4.67247,-74.1108
2,Carrera 48 93 64,ALPES BOGOTA HOSTAL,4.6814,-74.0612
3,Calle 10 2 40,APARTAESTUDIOS LA CANDELARIA,4.59482,-74.0718
4,Calle 57 18 26,CASA COLONIAL INN,4.64484,-74.0696
...,...,...,...,...
408,Carrera 63 98 66,WONDERFULL HOUSE 100,None,None
409,Calle 11 0 38,DESTINO NOMADA HOSTEL,None,None
410,Calle 12F 2 38,HOSTAL LA CANDELARIA BOGOTA,None,None
411,Calle 12F 2 55,HOSTAL SUE E.U.,None,None


## Nominatim: Open Street maps 

In [19]:
Direccion = 'Carrera 64 103C 16'
busqueda = Direccion + ', Bogotá, Colombia'

In [20]:
buscador_open = Nominatim(user_agent="pepito@dominio.com.co")
## ----- Busqueda ---- ######
Resultado = buscador_open.geocode(busqueda,timeout = 10000)

In [21]:
Resultado

Location(Carrera 64, Torcoroma, UPZ San Rafael, Localidad Puente Aranda, Bogotá, Bogotá Distrito Capital, Región Andina, 111621, Colombia, (4.6209197, -74.1225901, 0.0))

## Busquéda inversa: Dirección a partir de una latitud, longitud

In [22]:
cordenadas_buscar = str(Resultado.latitude)+','+str(Resultado.longitude)

In [23]:
cordenadas_buscar

'4.6209197,-74.1225901'

In [24]:
Inversa = buscador_open.reverse(cordenadas_buscar,timeout = 10000)

In [25]:
Inversa

Location(Panadería El Cacique, 2C-01, Carrera 64, Torcoroma, UPZ San Rafael, Localidad Puente Aranda, Bogotá, Bogotá Distrito Capital, Región Andina, 111621, Colombia, (4.6209382, -74.1225783, 0.0))

## Distancias entre kilometros

In [26]:
ML_centro_comercial = (4.602927,-74.065299)
SD = (4.603926,-74.065634)

print(geodesic(ML_centro_comercial, SD).kilometers)

0.116557303498864
